In [1]:
import findspark
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import matplotlib.pyplot as plt
import tensorflow as tf
from pyspark.sql import Row
import numpy as np
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
from pyspark.sql import SparkSession
findspark.init()

In [2]:
spark = SparkSession.builder.appName("DataCleaning").getOrCreate()

In [3]:
from pyspark import SparkFiles
file = "original.csv"
spark.sparkContext.addFile(file)
df = spark.read.csv(SparkFiles.get("original.csv"), sep=",", header=True)
df.createOrReplaceTempView('income_spark')

In [4]:
df.show()

+---+----------------+------+------------+-------------+------------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|   education|education_num|    marital_status|       occupation|  relationship| race|   sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+----------------+------+------------+-------------+------------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+------+
| 40|Self-emp-not-inc|223881| Prof-school|           15|Married-civ-spouse|   Prof-specialty|       Husband|White|  Male|       99999|           0|            70| United-States|  >50K|
| 30|         Private|149118|     HS-grad|            9|          Divorced|     Craft-repair| Not-in-family|White|Female|           0|           0|            40| United-States| <=50K|
| 46|         Private|109209|Some-college|           10|Married-civ-spouse|

In [5]:
#using spark.sql pull all the columsn from the df except for capital.gains, capital.loss, and fnlwgt
df = spark.sql("""SELECT age, 
               workclass, 
               education, 
               education_num, 
               marital_status, 
               occupation, 
               relationship, 
               race, 
               sex, 
               hours_per_week, 
               native_country, 
               income
               FROM income_spark""")

In [6]:
#make the spark dataframe df a pandas dataframe
income = df.toPandas()

In [7]:
#delete lines with missing values
income.replace("?", np.nan, inplace=True)
income.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income
0,40,Self-emp-not-inc,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,70,United-States,>50K
1,30,Private,HS-grad,9,Divorced,Craft-repair,Not-in-family,White,Female,40,United-States,<=50K
2,46,Private,Some-college,10,Married-civ-spouse,Adm-clerical,Husband,White,Male,40,United-States,>50K
3,32,Private,Assoc-voc,11,Married-civ-spouse,Other-service,Husband,White,Male,60,United-States,>50K
4,54,NaN,Preschool,1,Married-civ-spouse,NaN,Wife,White,Female,40,Mexico,<=50K


In [8]:
#remove all rows with NaN values
df = df.dropna()

In [9]:
#export and replace income.csv with the data in income
income.to_csv('income.csv', index=False)
